In [6]:
import covid19sim
import pandas as pd
import dill
from collections import defaultdict
import datetime

In [7]:
filename = "/Users/victor/Downloads/tracker_data_n_100_seed_0_20200504-233141.pkl"

In [8]:
data = dill.load(open(filename,"rb"))

In [9]:
humans = {}
for i in data['human_monitor'].keys():
    row = defaultdict(list)
    for c,j in enumerate(data['human_monitor'][i]):
        for k in j.keys():
            row[k].append(j[k])

    # index = (timestamp, attr)
    for k in row.keys():
        humans[(i,k)] = row[k]
    
humans = pd.DataFrame.from_dict(humans, orient="index")
humans.index = pd.MultiIndex.from_tuples(humans.index)

### Select chain of infections to track
Use `data['infection_monitor']` to select the chain of infections. If A infected B, B infected C. Note their numbers.

In [10]:
data['infection_monitor']

[['human:8', 'human:6', datetime.date(2020, 3, 1)],
 ['human:84', 'human:99', datetime.date(2020, 3, 1)],
 ['human:8', 'human:7', datetime.date(2020, 3, 1)],
 ['human:36', 'human:37', datetime.date(2020, 3, 2)],
 ['human:84', 'human:102', datetime.date(2020, 3, 2)],
 ['human:24', 'human:21', datetime.date(2020, 3, 2)],
 ['human:8', 'human:9', datetime.date(2020, 3, 3)],
 ['human:99', 'human:72', datetime.date(2020, 3, 4)],
 ['human:36', 'human:16', datetime.date(2020, 3, 4)],
 ['human:7', 'human:1', datetime.date(2020, 3, 4)],
 ['human:37', 'human:44', datetime.date(2020, 3, 5)],
 ['human:99', 'human:85', datetime.date(2020, 3, 5)],
 ['human:21', 'human:18', datetime.date(2020, 3, 5)],
 ['human:6', 'human:51', datetime.date(2020, 3, 5)],
 ['human:102', 'human:95', datetime.date(2020, 3, 5)],
 ['human:6', 'human:23', datetime.date(2020, 3, 6)],
 ['human:24', 'human:25', datetime.date(2020, 3, 6)],
 ['human:9', 'human:10', datetime.date(2020, 3, 7)],
 ['human:16', 'human:13', datetime.da

In [60]:
import matplotlib.pyplot as plt

def _color_cell(val):
    color = "white"
    if val == "positive":
        color = "red"
#     if val == 0.0:
#         color = "green"
    return f"background-color: {color}"

cmap = plt.get_cmap("cool", 16)

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % tuple([int(c*255)  for c in rgb[:3]])

RISK_COLOR = [f"background-color: {rgb_to_hex(cmap(k))}" for k in range(16)] + ["background-color: None"]
def _color_risk(human):
    return [
        RISK_COLOR[v] if k[1] == "risk_level" else RISK_COLOR[-1] for k, v in human.items()
    ]

In [61]:
RISK_COLOR

['background-color: #00ffff',
 'background-color: #11eeff',
 'background-color: #22ddff',
 'background-color: #33ccff',
 'background-color: #44bbff',
 'background-color: #55aaff',
 'background-color: #6699ff',
 'background-color: #7788ff',
 'background-color: #8877ff',
 'background-color: #9966ff',
 'background-color: #aa55ff',
 'background-color: #bb44ff',
 'background-color: #cc32ff',
 'background-color: #dd21ff',
 'background-color: #ee10ff',
 'background-color: #ff00ff',
 'background-color: None']

In [62]:
# track 8,6,51 (check from data['infection_monitor'])
pd.set_option('display.max_rows', None)
df = humans.loc[(slice(None), ("infection_timestamp","risk", "test_result", "n_symptoms", "risk_level")), [7,5,50]]
df.style.applymap(_color_cell).apply(_color_risk, axis=0)

# TODOs
# color test_results with 'red' and 'yellow' of lower transparency
# color risk_levels with one of 16 colors from a color map
# color recommendation_level with one of 4 colors - "red", "orange", "yellow", "green"
# have an option to select what to view i.e. test_results, symptoms, risk, risk_level, etc. 